In [1]:
import pandas as pd
import numpy as np
import _pickle as pickle
from collections import Counter
from tqdm import tqdm
import pymysql

In [2]:
negative_training_set = pickle.load(open('../data/negative_training_set.p', 'rb'))
positive_training_set = pickle.load(open('../data/positive_training_set.p', 'rb'))
negative_eval_set = pickle.load(open('../data/negative_eval_set.p', 'rb'))
positive_eval_set = pickle.load(open('../data/positive_eval_set.p', 'rb'))


In [3]:
diag_matrix = pickle.load(open('../data/diag_matrix.p', 'rb'))
diag_variables = pickle.load(open('../data/diag_variables.p', 'rb'))


In [4]:
negative_training_diag = np.array([diag_matrix[k] for k in negative_training_set])
positive_training_diag = np.array([diag_matrix[k] for k in positive_training_set])
negative_eval_diag = np.array([diag_matrix[k] for k in negative_eval_set])
positive_eval_diag = np.array([diag_matrix[k] for k in positive_eval_set])

In [5]:
# start tunnel first: ssh -f [uni]@mimir.dbmi.columbia.edu -L 3307:127.0.0.1:3306 -N

conn = pymysql.connect(host="127.0.0.1", 
                       user="", #uni
                       port = ,
                       passwd="", #sql password
                       db = "" ) #database
cur = conn.cursor()

In [6]:
all_visits_diag = {}

cur.execute('''select substring_index(icd10, '.', 1), count(distinct visit_id)
                from vfinal_1_predict_covid_conditions
                where substring_index(icd10, '.', 1) != ''
                group by substring_index(icd10, '.', 1);''')

for icd10, ct in cur.fetchall():
    all_visits_diag[icd10] = ct

In [7]:
visits_data = pickle.load(open('../data/visit_data.p', 'rb'))

visit_id_mrn = {}
for visit_id, pat_mrn_id in visits_data[:,(0,1)]:
    visit_id_mrn[visit_id] = pat_mrn_id
    

In [8]:
##table columns
##negative training set # positive training set # negative eval set # positive eval set # all visits

In [10]:
table_data = [[len(negative_training_set), len(positive_training_set), len(negative_eval_set), 
               len(positive_eval_set), 1573113]]
table_data

[[4670, 4670, 4670, 4670, 1573113]]

In [11]:
table_data.append([len(set([visit_id_mrn[k] for k in negative_training_set])),
                   len(set([visit_id_mrn[k] for k in positive_training_set])),
                   len(set([visit_id_mrn[k] for k in negative_eval_set])),
                   len(set([visit_id_mrn[k] for k in positive_eval_set])), 
                   434152])

table_data

[[4670, 4670, 4670, 4670, 1573113], [4606, 4178, 4592, 4137, 434152]]

In [12]:
for i in tqdm(range(len(diag_variables))):
    icd10_code = diag_variables[i]
    to_add = [Counter(negative_training_diag[:,i])[1],
              Counter(positive_training_diag[:,i])[1],
              Counter(negative_eval_diag[:,i])[1],
              Counter(positive_eval_diag[:,i])[1],
              all_visits_diag[icd10_code]]
    percent_to_add = [to_add[l]/table_data[0][l] for l in range(5)]
    table_data.append(to_add)
    table_data.append(percent_to_add)

100%|██████████| 1600/1600 [00:05<00:00, 284.96it/s]


In [13]:
table_columns = ['negative_training_set', 'positive_training_set', 'negative_eval_set', 'positive_eval_set', 'all_visits']
table_rows = ['N(visits)', 'N(patients)']
for k in diag_variables:
    table_rows += [('N_' + k), ('%_' + k)]

In [14]:
pd.DataFrame(table_data, columns=table_columns, index=table_rows).to_csv('table_s2_diag.csv')